In [189]:
import json
import pennylane as qml
import pennylane.numpy as np

In [269]:
np.random.seed(1967)

def get_matrix(params):
    """
    Args:
        - params (array): The four parameters of the model.
        
    Returns:
        - (matrix): The associated matrix to these parameters.
    """
    alpha, beta, gamma, phi = params
    #print(params,'test2')
    # Put your code here #
   
    rz_gamma = qml.matrix(qml.RZ(gamma, wires=0))
    rx_beta = qml.matrix(qml.RX(beta, wires=0))
    rz_alpha = qml.matrix(qml.RZ(alpha, wires=0))

    # Return the matrix
    return np.exp(1j * phi) * rz_gamma @ rx_beta @ rz_alpha

In [284]:
def error(U, params):
    """
    This function determines the similarity between your generated matrix and
    the target unitary.

    Args:
        - U (np.array): Goal matrix that we want to approach.
        - params (array): The four parameters of the model.

    Returns:
        - (float): Error associated with the quality of the solution.
    """

    matrix = get_matrix(params)
    # Put your code here #
    error =  np.abs(U - matrix).sum()
    #print(error,'test1')
    # Return the error
    return error

In [285]:
def train_parameters(U):
    epochs = 2000
    lr = 0.01

    grad = qml.grad(error, argnum=1)
    params = np.random.rand(4) * np.pi

    for epoch in range(epochs):
        params -= lr * grad(U, params)
        #print(epoch, params)

    return params

In [286]:
# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    matrix = json.loads(test_case_input)
    params = [float(p) for p in train_parameters(matrix)]
    #print(params,'break')
    return json.dumps(params)


def check(solution_output: str, expected_output: str) -> None:
    matrix1 = get_matrix(json.loads(solution_output))
    matrix2 = json.loads(expected_output)
    print(matrix1,'final matrix')
    assert not np.allclose(get_matrix(np.random.rand(4)), get_matrix(np.random.rand(4)))
    assert np.allclose(matrix1, matrix2, atol=0.2)

In [287]:
# These are the public test cases
test_cases = [
    ('[[ 0.70710678,  0.70710678], [ 0.70710678, -0.70710678]]', '[[ 0.70710678,  0.70710678], [ 0.70710678, -0.70710678]]'),
    ('[[ 1,  0], [ 0, -1]]', '[[ 1,  0], [ 0, -1]]')
]

In [288]:
# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[[ 0.70710678,  0.70710678], [ 0.70710678, -0.70710678]]'...
[[ 0.70695158+0.01481417j  0.70703181+0.01029652j]
 [ 0.70684249+0.01933119j -0.70695158-0.01481415j]] final matrix
Correct!
Running test case 1 with input '[[ 1,  0], [ 0, -1]]'...
[[ 0.99999625-0.00067668j  0.00263044+0.00034372j]
 [ 0.00263044-0.00034372j -0.99999625-0.00067668j]] final matrix
Correct!
